In [ ]:
import pandas as pd 
from pathlib import Path 
import math, random 
import torch 
import torchaudio 
from torchaudio import transforms 
from IPython.display import Audio 
from torch.utils.data import DataLoader, Dataset, random_split 
import torch.nn.functional as F 
from torch.nn import init 
import torch.nn as nn
import numpy as np
from sklearn.metrics import f1_score
import glob, os
from scipy.io import wavfile
import csv
from sklearn.model_selection import train_test_split

os.chdir("D:\Pytorch_train");
#读取csv文件 
download_path = Path.cwd()/'hj_data/train' 
noise_dir = Path.cwd()/'free-sound'
# Read metadata file 
metadata_file = download_path/'data.csv' 
traindata_file = download_path/'train.csv'
evaldata_file = download_path/'eval.csv'
df = pd.read_csv(metadata_file) 
df.head() 
 
# Construct file path by concatenating fold and file name 
df['relative_path'] = '/' + df['fold'].astype(str) + '/' + df['slice_file_name'].astype(str) 
# train_df['relative_path'] = '/' + df['fold'].astype(str) + '/' + df['slice_file_name'].astype(str) 
# eval_df['relative_path'] = '/' + df['fold'].astype(str) + '/' + df['slice_file_name'].astype(str) 
 
# Take relevant columns 
df = df[['relative_path', 'classID']] 
# train_df = train_df[['relative_path', 'classID']] 
# eval_df = eval_df[['relative_path', 'classID']] 
df.head()

#读取文件中的音频
# import math, random 
# import torch 
# import torchaudio 
# from torchaudio import transforms 
# from IPython.display import Audio 
 
class AudioUtil(): 
  # ---------------------------- 
  # Load an audio file. Return the signal as a tensor and the sample rate 
  # ---------------------------- 
  @staticmethod 
  #audio_file = download_path/'fold1'/'101415-3-0-2.wav'
  def open(audio_file): 
    sig, sr = torchaudio.load(audio_file) 
    return (sig, sr)

#转换为立体声
# ---------------------------- 未执行
  # Convert the given audio to the desired number of channels 
  # ---------------------------- 
  @staticmethod 
  def rechannel(aud, new_channel): 
    sig, sr = aud 
 
    if (sig.shape[0] == new_channel): 
      # Nothing to do 
      return aud 
 
    if (new_channel == 1): 
      # Convert from stereo to mono by selecting only the first channel 
      resig = sig[:1, :] 
    else: 
      # Convert from mono to stereo by duplicating the first channel 
      resig = torch.cat([sig, sig]) 
 
    return ((resig, sr))

#标准化采样率
# ---------------------------- 
  # Since Resample applies to a single channel, we resample one channel at a time 
  # ---------------------------- 
  @staticmethod 
  def resample(aud, newsr): 
    sig, sr = aud 
 
    if (sr == newsr): 
      # Nothing to do 
      return aud 
 
    num_channels = sig.shape[0] 
    # Resample first channel 
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:]) 
    if (num_channels > 1): 
      # Resample the second channel and merge both channels 
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:]) 
      resig = torch.cat([resig, retwo]) 
 
    return ((resig, newsr))

#调整为相同长度
# ---------------------------- 
  # Pad (or truncate) the signal to a fixed length 'max_ms' in milliseconds 
  # 使用ms为单位进行固定长度，而不是帧长
  @staticmethod 
  def pad_trunc(aud, max_ms): 
    sig, sr = aud 
#     print("sig_shape:".format(sig.shape))
    num_rows, sig_len = sig.shape 
#     print(f"sig_len:{sig_len}")
    max_len = sr//1000 * max_ms 
 
    if (sig_len > max_len): 
      # Truncate the signal to the given length 
      sig = sig[:,:max_len] 
 
    elif (sig_len < max_len): 
      # Length of padding to add at the beginning and end of the signal 
      pad_begin_len = random.randint(0, max_len - sig_len) 
      pad_end_len = max_len - sig_len - pad_begin_len 
 
      # Pad with 0s 
      pad_begin = torch.zeros((num_rows, pad_begin_len)) 
      pad_end = torch.zeros((num_rows, pad_end_len)) 
 
      sig = torch.cat((pad_begin, sig, pad_end), 1) 
#     print(f"sig_shape_after:{sig.shape}")
    return (sig, sr)

#数据扩充增广（时移）
# ---------------------------- 
  # Shifts the signal to the left or right by some percent. Values at the end 
  # are 'wrapped around' to the start of the transformed signal. 
  # ---------------------------- 
  @staticmethod 
  def time_shift(aud, shift_limit): 
    sig,sr = aud 
#     print(f"afterpad_sig_shape:{sig.shape}")#[2, 220000]
    _, sig_len = sig.shape 
    shift_amt = int(random.random() * shift_limit * sig_len) 
    return (sig.roll(shift_amt), sr)

#梅尔谱图
# ---------------------------- 
  # Generate a Spectrogram 
  # ---------------------------- 
  @staticmethod 
  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None): 
    sig,sr = aud 
#     print(f"after_timeshift_sig_shape:{sig.shape}")#[2, 220000]维度不变
    top_db = 80 
 
    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc 
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig) 
#     print(f"spec_shape:{spec.shape}")#[2, 64, 430] n_mels=64, n_fft=1024, hop_len=None
    # Convert to decibels 
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec) 
    return (spec)
  
#   @staticmethod 
#   def GaussNoise(spectrogram):
#       if np.random.random() > self.add_noise_prob:
#           return spectrogram

#       # set the std value 
#       min_pixel_value = np.min(spectrogram)
#       if self.std is None:
#         std_factor = 0.03     # factor number 
#         std = np.abs(min_pixel_value*std_factor)

#       # generate a white noise spectrogram
#       gauss_mask = np.random.normal(self.mean, 
#                                     std, 
#                                     size=self.input_size).astype('float32')
      
#       # add white noise to the sound spectrogram
#       noisy_spectrogram = spectrogram + gauss_mask

#       return noisy_spectrogram

#数据扩充：时间和频率屏蔽
# ---------------------------- 
  # Augment the Spectrogram by masking out some sections of it in both the frequency 
  # dimension (ie. horizontal bars) and the time dimension (vertical bars) to prevent 
  # overfitting and to help the model generalise better. The masked sections are 
  # replaced with the mean value. 
  # ---------------------------- 
  @staticmethod 
  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1): 
    _, n_mels, n_steps = spec.shape 
    mask_value = spec.mean() 
    aug_spec = spec 
 
    freq_mask_param = max_mask_pct * n_mels 
    for _ in range(n_freq_masks): 
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value) 
 
    time_mask_param = max_mask_pct * n_steps 
    for _ in range(n_time_masks): 
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value) 
 
    return aug_spec

  #加噪（musan）
  @staticmethod 
  def noise_augmentation(wave, noise_dir):
    """ Perform noise augmentation of the wave by loading three noise segments
    from the noise_dir and add these on top of the wave with a dampening factor
    of 0.4
    """
    noise_paths = glob.glob(os.path.join(noise_dir, "*.wav"))
    aug_noise_paths = np.random.choice(noise_paths, 1, replace=False)
    print(aug_noise_paths)
    aug_noise_paths = str(aug_noise_paths[0])
    print(aug_noise_paths)
    #(fs, aug_noise) = wavfile.read(aug_noise_path)
    aug_noise = AudioUtil.open(aug_noise_paths)
    aug_noise = AudioUtil.resample(aug_noise, 22050) #标准化采样率
    aug_noise = AudioUtil.rechannel(aug_noise, 2) #转换为立体声，统一为两个声道
    aug_noise = AudioUtil.pad_trunc(aug_noise, 10000) #调整为相同长度
    #print(f'wave:{wave}, aug_noise:{aug_noise}')
    dampening_factor = 0.4
    for aug_noise_path in aug_noise_paths:
        aug_noise = list(aug_noise)
        wave = list(wave)
#         initial_wave = np.concatenate(wave)
        
#         print(f'wave1:{wave}, aug_noise1:{aug_noise}')
#         aug_noise = torch.tensor([item.detach().numpy() for item in aug_noise])
        wave[0] = wave[0] + aug_noise[0]*dampening_factor
#         print(f'wave2:{wave}, aug_noise2:{aug_noise}')
#         wave = torch.from_numpy(wave)
#         print(f'wave:{wave}')
    return wave

#自定义数据加载器
# from torch.utils.data import DataLoader, Dataset, random_split 
# import torchaudio 
 
# ---------------------------- 
# Sound Dataset 
# ---------------------------- 
# SoundDS：MyDataset类


class SoundDS(Dataset): 
  def __init__(self, df, data_path): 
    self.df = df #df为csv文件
    self.data_path = str(data_path) 
    self.duration = 10000 #固定长度，单位为ms
    self.sr = 22050 #采样率
    self.channel = 2 #通道数
    self.shift_pct = 0.4        
 
  # ---------------------------- 
  # Number of items in dataset 
  # ---------------------------- 
  def __len__(self): 
    return len(self.df)     
 
  # ---------------------------- 
  # Get i'th item in dataset 
  # ---------------------------- 
  def __getitem__(self, idx): 
    # Absolute file path of the audio file - concatenate the audio directory with 
    # the relative path 
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']  #.loc:取idx对应行的所有数据，.loc[idx, 'relative_path']:取idx对应行的relative_path
#     print(audio_file)
    # Get the Class ID 
    class_id = self.df.loc[idx, 'classID'] #取idx对应行的classID
 
    aud = AudioUtil.open(audio_file) 
#     print(f'aud:{aud}')
#     AudioUtil.show_wave(aud)
    # 有些声音有更高的采样率，或者比大多数声音更少的通道。所以让所有声音都有相同数量的通道和相同的采样率。除非采样速率相同，否则pad_trunc仍然会产生不同长度的数组，即使声音持续时间相同。
    reaud = AudioUtil.resample(aud, self.sr) #标准化采样率
    rechan = AudioUtil.rechannel(reaud, self.channel) #转换为立体声，统一为两个声道
    dur_aud = AudioUtil.pad_trunc(rechan, self.duration) #调整为相同长度
#     print(dur_aud)
#     print(f'dur_aud:{dur_aud}')
#     noise_aud = AudioUtil.noise_augmentation(dur_aud, noise_dir) #加噪
#     print(f'noist_aud:{noise_aud}')
    shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct) #时移
#     print(f'shift_aud:{shift_aud}')
    sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None) #转换为mel谱图
#     print(f'sgram:{sgram}')
    
    aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2) #时间和频率屏蔽
#     print(f'aug_agram:{aug_sgram}')
#     print("mel-length:{}".format(aug_sgram.shape))
#     print(aug_sgram)
#     #画mel谱图
#     librosa.display.specshow(mel_spect, sr=fs, x_axis='time', y_axis='mel')
#     plt.ylabel('Mel Frequency')
#     plt.xlabel('Time(s)')
#     plt.title('Mel Spectrogram')
#     plt.show()
    
    return aug_sgram, class_id

class eval_SoundDS(Dataset): 
  def __init__(self, df, data_path): 
    self.df = df #df为csv文件
    self.data_path = str(data_path) 
    self.duration = 10000 #固定长度，单位为ms
    self.sr = 22050 #采样率
    self.channel = 2 #通道数
    self.shift_pct = 0.4        
 
  # ---------------------------- 
  # Number of items in dataset 
  # ---------------------------- 
  def __len__(self): 
    return len(self.df)     
 
  # ---------------------------- 
  # Get i'th item in dataset 
  # ---------------------------- 
  def __getitem__(self, idx): 
    # Absolute file path of the audio file - concatenate the audio directory with 
    # the relative path 
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']  #.loc:取idx对应行的所有数据，.loc[idx, 'relative_path']:取idx对应行的relative_path
    # Get the Class ID 
    class_id = self.df.loc[idx, 'classID'] #取idx对应行的classID
 
    aud = AudioUtil.open(audio_file) 
    # 有些声音有更高的采样率，或者比大多数声音更少的通道。所以让所有声音都有相同数量的通道和相同的采样率。除非采样速率相同，否则pad_trunc仍然会产生不同长度的数组，即使声音持续时间相同。
    reaud = AudioUtil.resample(aud, self.sr) #标准化采样率
    rechan = AudioUtil.rechannel(reaud, self.channel) #转换为立体声，统一为两个声道
    dur_aud = AudioUtil.pad_trunc(rechan, self.duration) #调整为相同长度
    sgram = AudioUtil.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None) #转换为mel谱图
   
    return sgram, class_id

#使用数据加载器准备一批数据
from torch.utils.data import random_split 

from pathlib import Path 
download_path = Path.cwd()/'hj_data/train' 

metadata_file = download_path/'data.csv' 
traindata_file = download_path/'data.csv'
evaldata_file = download_path/'val.csv'
# def read_data(test_data=download_path/'data.csv', n=0, label=1):
#     '''
#     加载数据的功能
#     n:特征数据起始位
#     label：是否是监督样本数据
#     '''
#     csv.field_size_limit(500 * 1024 * 1024) #一定要加上这一句
#     csv_reader = csv.reader(open(test_data, encoding="utf8", errors="ignore"))
#     data_list = []
#     for one_line in csv_reader:
#         data_list.append(one_line)
#     x_list = []
#     y_list = []
#     for one_line in data_list[1:]:
#         if label == 1:#如果是监督样本数据
#             y_list.append(int(one_line[-1]))  # 标志位(最后一位都是标签位)
#             one_list = [o for o in one_line[n:-1]]
#             x_list.append(one_list)
#         else:
#             one_list = [o for o in one_line[n:]]
#             x_list.append(one_list)
#     return x_list, y_list
 
# def split_data(data_list, y_list, ratio=0.10):#70%训练集，30%测试集: 914285,391837
#     '''
#     按照指定的比例，划分样本数据集
#     ratio: 测试数据的比率
#     '''
#     X_train, X_test, y_train, y_test = train_test_split(data_list, y_list, test_size=ratio, random_state=50)
 
#     """训练集"""
#     if traindata_file == True:
#             traindata_file.unlink()  ## 删除已经存在的train.csv文件 (方便下一个epoch重新划分数据集)
#     with open(traindata_file, 'w', encoding="utf8",newline="", errors="ignore") as csvfile:#不加newline=""的话会空一行出来
#         fieldnames = ['slice_file_name', 'fold','class','classID']
#         write = csv.DictWriter(csvfile,fieldnames=fieldnames)
#         write.writeheader()#写表头
#         for i in range(len(X_train)):
#            write.writerow({'slice_file_name':X_train[i][0],'fold':X_train[i][1],'class':X_train[i][2],'classID':y_train[i]})
 
#     """测试集"""
#     #标签文件
# #     with open('input/sub_test_y', 'w') as fp:
# #         json.dump(y_test, fp)
#     #测试csv
#     if evaldata_file == True:
#             evaldata_file.unlink()  ## 删除已经存在的train.csv文件 (方便下一个epoch重新划分数据集)
#     with open(evaldata_file, 'w', encoding="utf8",newline="", errors="ignore") as csvfile:#不加newline=""的话会空一行出来
#         fieldnames = ['slice_file_name', 'fold','classID','class']
#         write = csv.DictWriter(csvfile,fieldnames=fieldnames)
#         write.writeheader()#写表头
#         for i in range(len(X_test)):
#            write.writerow({'slice_file_name':X_train[i][0],'fold':X_train[i][1],'classID':X_train[i][2],'classID':y_train[i]})
#     return X_train, X_test, y_train, y_test
 

# """获取大文件的数据"""
# x_list, y_list=read_data()
# """划分为训练集和测试集及label文件"""
# split_data(x_list,y_list)

 
train_df = pd.read_csv(traindata_file)
# if traindata_file==NULL or True:
#     """获取大文件的数据"""
#     x_list, y_list=read_data()
#     """划分为训练集和测试集及label文件"""
#     split_data(x_list,y_list)
    
train_df.head() 

train_df['relative_path'] = '/' + train_df['fold'].astype(str) + '/' + train_df['slice_file_name'].astype(str) 
train_df = train_df[['relative_path', 'classID']] 

eval_df = pd.read_csv(evaldata_file) 
eval_df.head()

eval_df['relative_path'] = '/' + eval_df['fold'].astype(str) + '/' + eval_df['slice_file_name'].astype(str) 

eval_df = eval_df[['relative_path', 'classID']] 

train_ds = SoundDS(train_df,download_path)
# train_ds2 = eval_SoundDS(train_df,download_path) 
# train_ds.Merge(train_ds2,true,MissingSchemaAction.AddWithKey);

eval_ds = eval_SoundDS(eval_df,download_path) 
 
 
# Create training and validation data loaders 
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True) 
val_dl = torch.utils.data.DataLoader(eval_ds, batch_size=64, shuffle=True)



#建立模型
# import torch.nn.functional as F 
# from torch.nn import init 
 
# ---------------------------- 
# Audio Classification Model 
# ---------------------------- 
class AudioClassifier (nn.Module): 
    # ---------------------------- 
    # Build the model architecture 
    # ---------------------------- 
    def __init__(self): 
        super().__init__() 
        conv_layers = [] 
 
        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization 
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)) #in_channels=2,out_channels=8
        self.relu1 = nn.PReLU() 
        self.bn1 = nn.BatchNorm2d(8) 
        init.kaiming_normal_(self.conv1.weight, a=0.1) #a-此层之后使用的整流器的负斜率(仅与 'leaky_relu' 一起使用)
        self.conv1.bias.data.zero_() 
        conv_layers += [self.conv1, self.relu1, self.bn1] 
 
        # Second Convolution Block 
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)) 
        self.relu2 = nn.PReLU() 
        self.bn2 = nn.BatchNorm2d(16) 
        init.kaiming_normal_(self.conv2.weight, a=0.1) 
        self.conv2.bias.data.zero_() 
        conv_layers += [self.conv2, self.relu2, self.bn2] 
 
        # third Convolution Block 
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)) 
        self.relu3 = nn.PReLU() 
        self.bn3 = nn.BatchNorm2d(32) 
        init.kaiming_normal_(self.conv3.weight, a=0.1) 
        self.conv3.bias.data.zero_() 
        conv_layers += [self.conv3, self.relu3, self.bn3] 
 
        # fourth Convolution Block 
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)) 
        self.relu4 = nn.PReLU() 
        self.bn4 = nn.BatchNorm2d(64) 
        init.kaiming_normal_(self.conv4.weight, a=0.1) 
        self.conv4.bias.data.zero_() 
        conv_layers += [self.conv4, self.relu4, self.bn4] 
         
        # fifth Convolution Block 
        self.conv5 = nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)) 
        self.relu5 = nn.PReLU() 
        self.bn5 = nn.BatchNorm2d(128) 
        init.kaiming_normal_(self.conv4.weight, a=0.1) 
        self.conv5.bias.data.zero_() 
        conv_layers += [self.conv5, self.relu5, self.bn5] 
        
        # sixth Convolution Block 
        self.conv6 = nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)) 
        self.relu6 = nn.PReLU() 
        self.bn6 = nn.BatchNorm2d(256) 
        init.kaiming_normal_(self.conv6.weight, a=0.1) 
        self.conv6.bias.data.zero_() 
        conv_layers += [self.conv6, self.relu6, self.bn6] 
        
        # Linear Classifier 
        self.ap = nn.AdaptiveAvgPool2d(output_size=1) 
        self.lin = nn.Linear(in_features=256, out_features=6) 
        
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Conv2d(256,256//16,kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(256//16,256,kernel_size=1),
            nn.Sigmoid()
        )

 
        # Wrap the Convolutional Blocks 打包卷积块
        self.conv = nn.Sequential(*conv_layers) 
 
    # ---------------------------- 
    # Forward pass computations 
    # ---------------------------- 
    def forward(self, x): 
        # Run the convolutional blocks 
        x = self.conv(x) 
        
        x_se = self.se(x)
        x = x * x_se
 
        # Adaptive pool and flatten for input to linear layer 
        x = self.ap(x) 
        x = x.view(x.shape[0], -1) #=reshape 重新定义矩阵形状，-1代表动态调整这个维度上的元素个数，以保证元素的总数不变
 
        # Linear layer 
        logits1 = self.lin(self.dropout1(x))
        logits2 = self.lin(self.dropout2(x))
        logits3 = self.lin(self.dropout3(x))
        logits4 = self.lin(self.dropout4(x))
        logits5 = self.lin(self.dropout5(x))

        x = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
#         x = self.lin(x) 
 
        # Final output 
        return x 
 
# Create the model and put it on the GPU if available 
myModel = AudioClassifier() 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device) 
# Check that it is on Cuda 
next(myModel.parameters()).device

#训练
# ---------------------------- 
# Training Loop 
# ---------------------------- 
def training(model, train_dl, num_epochs): 
  # Loss Function, Optimizer and Scheduler 
  
  def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    # device = torch.device('cpu')
    # # preds = preds.to(device)
    # # truths = preds.to(device)
    # preds.cpu()
    # truths.cpu()
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    return f1_score(truths, preds,average='micro')

#   def logf1_loss(truths,preds):
#         f1_score = micro_f1(preds, truths)
#         loss = -(np.log(f1_score))
#         loss = torch.tensor([loss], requires_grad=True)
#         return loss



  criterion = nn.CrossEntropyLoss() 
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001) 
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, 
                                                steps_per_epoch=int(len(train_dl)), 
                                                epochs=num_epochs, 
                                                anneal_strategy='linear') #退火策略，'cos'和'liner'分别表示余弦退火和线性退火
 
  expdir = Path.cwd()/'hj_data/experiments'
  version = 'v11'
  logfilepath = expdir / version / 'logs_{}.txt'.format(version)
  logfilepath_eval = expdir / version / 'logs_eval_{}.txt'.format(version)
  model_savedir = expdir / version 
  def init_logger(self):
        self.logFileName = str(self.logfilepath)
        if self.logfilepath.exists == True:
            self.logfilepath.exists.unlink()  ## 删除已经存在的log文件 (方便重启实验)
        if self.logfilepath_eval.exists == True:
            self.logfilepath_eval.exists.unlink()  ## 删除已经存在的log文件 (方便重启实验)
        self.logFileName_eval = str(self.logfilepath_eval)

        ## 存储training log文件
        with open(self.logFileName, 'a', encoding='utf-8') as wf:
            # #将参数类中的所有参数写入log
            # for k, v in self.hp.__dict__.items():
            #     wf.write("{} : {}\n".format(k, v))
            # wf.write('-' * 50 + "Experiment & Hparams Created" + "-" * 50 + "\n")
            wf.write("*" * 100 + "\n")
            wf.close()

    ## 该方法将一个字典  ，按kv的顺序，写入一行到 log.txt
  def write_line2log(log_dict: dict, filedir, isprint: True):
      strp = ''
      with open(filedir, 'a', encoding='utf-8') as f:
          for key, value in log_dict.items():
              witem = '{}'.format(key) + ':{},'.format(value)
              strp += witem
          f.write(strp)
          #f.write('当前进程的内存使用：%.4f GB' % (psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024 / 1024))
          f.write('\n')
      if isprint:
          print(strp)
      pass

  def print_network(model, name):
      """Print out the network information."""
      num_params = 0
      for p in model.parameters():
          num_params += p.numel()
      print("Model {},the number of parameters: {}".format(name, num_params))

  #保存模型
  def save_model(i):
      pdict = {"model":model.state_dict(),
                }
      path = model_savedir / "{:04}.pth".format(i)
      torch.save(pdict, str(path))
      print("---------------- model saved ------------------- ")
  


  #测试
  def test_acc (model, val_dl):
    correct_prediction = 0
    total_prediction = 0

    truths = []
    preds = []
    # Disable gradient updates
    with torch.no_grad():
      for data in val_dl:
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = data[0].to(device), data[1].to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Get predictions
        outputs = model(inputs)

        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs,1)
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]
#         print(f'labes: {labels},preds: {prediction}')
        # state = torch.device('cpu')
        # labels = labels.to(device)
        # prediction = prediction.to(device)
        labels = labels.cpu()
        prediction = prediction.cpu()
#         print(f'labes: {labels},preds: {prediction}')
        truths.append(labels)
        preds.append(prediction)

    
    acc = correct_prediction/total_prediction
    f1 = micro_f1(truths,preds)
    losse_curves  = {"eval_step--":"",
                    "Epoch":epoch,
                    "Accuracy":acc,
                    "Micro_f1":f1,
                    "Total items":total_prediction}
    write_line2log(losse_curves, logfilepath_eval, isprint=True)


  # Repeat for each epoch 
  for epoch in range(1,num_epochs): 
    running_loss = 0.0 
    correct_prediction = 0 
    total_prediction = 0 
 
    # Repeat for each batch in the training set 
    for i, data in enumerate(train_dl):
#         data[0] = list(data[0])
#         data[0][1] = data[0][1].item()
#         data[0][0] = data[0][0].numpy().tolist()
#         data[0][0][0] = list(data[0][0][0])
#         print(data[0])
#         if i%2 == 0:
#             data[0] = AudioUtil.noise_augmentation(data[0], noise_dir) #加噪
#             data[0] = AudioUtil.time_shift(data[0], 0.4) #时移
#             #     print(f'shift_aud:{shift_aud}')
#             data[0] = AudioUtil.spectro_gram(data[0], n_mels=64, n_fft=1024, hop_len=None) #转换为mel谱图
#             #     print(f'sgram:{sgram}')
#             data[0] = AudioUtil.spectro_augment(data[0], max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2) #时间和频率屏蔽
#         else:
#             data[0] = AudioUtil.spectro_gram(data[0], n_mels=64, n_fft=1024, hop_len=None) #转换为mel谱图
        # Get the input features and target labels, and put them on the GPU 
        inputs, labels = data[0].to(device), data[1].to(device) 
 
        # Normalize the inputs 
        inputs_m, inputs_s = inputs.mean(), inputs.std() #标准差
        inputs = (inputs - inputs_m) / inputs_s 
 
        # Zero the parameter gradients 
        optimizer.zero_grad() 
 
        # forward + backward + optimize 
        outputs = model(inputs) 
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
        scheduler.step() 
 
#         # Keep stats for Loss and Accuracy 
#         running_loss += loss.item() 
 
        # Get the predicted class with the highest score 
        _, prediction = torch.max(outputs,1) 
        # Count of predictions that matched the target label 
        correct_prediction += (prediction == labels).sum().item() 
        total_prediction += prediction.shape[0]
        
#         truths = []
#         preds = []
#         labels = labels.cpu()
#         prediction = prediction.cpu()
# #         print(f'labes: {labels},preds: {prediction}')
#         truths.append(labels)
#         preds.append(prediction)
#         loss2 = logf1_loss(truths, preds) 
#         loss1.backward() 
#         optimizer.step() 
#         scheduler.step() 
 
        # Keep stats for Loss and Accuracy 
        running_loss += loss.item()
 
        # if i % 10 == 0:    # print every 10 mini-batches 
        #     print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10)) 
 
    # Print stats at the end of the epoch 
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches 
    acc = correct_prediction/total_prediction 
    losse_curves  = {"Epoch":epoch,
                            "Loss":avg_loss,
                             "Accuracy":acc,
                             }
    if epoch == 1:
      print("create loss dict")
      loss_log_dict = {}
      for k, v in losse_curves.items():
        loss_log_dict[k] = []
      print("loss dict created")
    
    for k, v in loss_log_dict.items():
      loss_log_dict[k].append(losse_curves[k])  # 把每batch的loss数据加入到 loss curves中
    write_line2log(losse_curves, logfilepath, isprint=True)

    ## 模型保存
    if epoch % 50 == 0:
      test_acc(myModel,val_dl)
      save_model(epoch)
    # print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}') 
 
  print('Finished Training') 
 
num_epochs=501   # Just for demo, adjust this higher. 
# """获取大文件的数据"""
# x_list, y_list=read_data()
# """划分为训练集和测试集及label文件"""
# split_data(x_list,y_list)
training(myModel, train_dl, num_epochs)
# if __name__ == '__main__':
#     """获取大文件的数据"""
#     x_list, y_list=read_data()
#     """划分为训练集和测试集及label文件"""
#     split_data(x_list,y_list)
#     training(myModel, train_dl, num_epochs)

D:\Anaconda\envs\pytorch\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


create loss dict
loss dict created
Epoch:1,Loss:1.7629191743002997,Accuracy:0.26181818181818184,
Epoch:2,Loss:1.705138921737671,Accuracy:0.29818181818181816,
Epoch:3,Loss:1.6527327564027574,Accuracy:0.4,
Epoch:4,Loss:1.5989306105507746,Accuracy:0.4381818181818182,
Epoch:5,Loss:1.5412685208850436,Accuracy:0.4690909090909091,
Epoch:6,Loss:1.4965039491653442,Accuracy:0.4709090909090909,
Epoch:7,Loss:1.453702449798584,Accuracy:0.48727272727272725,
Epoch:8,Loss:1.4006497197681003,Accuracy:0.5272727272727272,
Epoch:9,Loss:1.3635508881674872,Accuracy:0.5363636363636364,
Epoch:10,Loss:1.312432262632582,Accuracy:0.5309090909090909,
Epoch:11,Loss:1.261875483724806,Accuracy:0.5363636363636364,
Epoch:12,Loss:1.238268772761027,Accuracy:0.5709090909090909,
Epoch:13,Loss:1.185283899307251,Accuracy:0.5781818181818181,
Epoch:14,Loss:1.1392408344480727,Accuracy:0.610909090909091,
Epoch:15,Loss:1.1061904297934637,Accuracy:0.5945454545454546,
Epoch:16,Loss:1.0728545718722873,Accuracy:0.6218181818181818,
E

In [5]:
import torchvision.models
import torch
import torchsummary

model = torchvision.models.vgg16(pretrained=False)
device = torch.device('cpu')
model.to(device)

torchsummary.summary(model.cuda(), (3, 224, 224))

D:\Anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [6]:
total = sum([param.nelement() for param in model.parameters()])
print("Number of parameter: %.2fM" % (total/1e6))

Number of parameter: 138.36M


In [ ]:

import nessi

model = myModel
nessi.get_model_size(model, 'torch' ,input_size=(32,2,64,430))